In [ ]:
import torch
import torchvision
import torch.nn as nn 
import numpy as np
import torchvision.transforms as transforms 
import matplotlib.pyplot as plt

#우리가 사용할 컴퓨터를 check하는 부분, cpu/gpu 지원받을지..
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# 하이퍼파라미터 설정

input_size = 784 # mnist는 28x28x1로 구성
hidden_size = 500 # 은닉층의 unit 수
num_classes = 10 # 카테고리 개수
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# dataset 로딩 - 2개의 과정을 거친다
# train_dataset, test_dataset 데이터 로딩 1단계
train_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train = True, #traindataset만 저장
                                           transform = transforms.ToTensor(),
                                           download = True) #traindataset을 스케일링하여 다운로드하겠다.
test_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train = False, #test_dataset만 저장, test옵션은 따로 없음.
                                           transform = transforms.ToTensor())

# 데이터 로딩 2단계.. BatchSize 이용
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size = batch_size,
                                           shuffle =True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size = batch_size,
                                           shuffle =False) #테스트는 학습이아니므로 shuffle 필요 없다.

#### Model 생성


In [ ]:
class NeuralNet(nn.Module): #nn.Module을 상속받아 NeuralNet이라는 클래스 생성
    # 모델 설계
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) # Linear : Wx+b이므로
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    # 모델의 Forward Path 정의
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)

        return out #여기까지 클래스 정의 부분

# 위에서 정의한 클래스를 인스턴스화 시킴
model = NeuralNet(input_size, hidden_size, num_classes).to(device) # to(device).. 이 모델을 gpu서버에서 돌리겠다.

# loss and optimizer를 선정의
loss_function = nn.CrossEntropyLoss() # Loss 기능 안에 Softmax(예측값 극대화 후 확률값으로 변환)함수 기능 포함되어져 있음
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step=len(train_loader) # 600번 로딩.. 600번 학습 진행

for epoch in range(num_epochs): # 5번...
    for i,(images, labels) in enumerate(train_loader): # 한번에 100개씩 총 600번 들어가고 그게 5번(num_epochs) 반복!!!
        #네트워크에 넣어줄때는 1차원으로 펼쳐서 넣는다.
        images = images.reshape(-1, 28*28).to(device) #-1은 나머지 즉, 채널을 의미, gpu로 돌린다.
        labels = labels.to(device)

        # Forward Pass
        pred = model(images)
        loss = loss_function(pred, labels)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        if(i+1) % 100 ==0:
            print('Epoch[{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [ ]:
# Test
'''
1) with torch.no_grad():
    some code
    --> backward를 하지 않겠다.

2) with torch.grad_enabled(False):

만약 이 구문을 하지 않으면
기본적으로 BackPropagation을 진행할 것으로 알고 이에 해당하는 
메모리를 따로 빼둔다.

결과적으로 이 구문을 작성하지 않으면 메모리를 더 많이 먹는다.


'''
with torch.no_grad(): # 실제로 학습할 필요 없을 때 이 구문을 반드시 작성
    correct = 0
    total = 0

    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        outputs = model(images)
        '''
        max는 기본적으로 2개의 리턴 값을 가짐
            - 결과에 해당하는 값 
            - 값에 해당하는 인덱스 리턴
        _(언더바)로 변수를 받을 경우 값을 리턴 받겠지만 필요없으므로 값을 저장하지는 않겠다.
        1을 사용하지 않으면 디폴트는 0인데, 디폴트로 그냥 둘 경우
        세로로 값을 비교한다. 가로값과 비교하기위해 1을 사용한다. 
        '''
        _,predicted = torch.max(outputs.data, 1)

        total +=labels.size(0)
        correct +=(predicted == labels).sum().item()
    

    print('Accuracy of the Network on the Test Images :{} %'.format(100 * correct/total))
torch.save(model.state_dict(), 'model.ckpt')